In [1]:
import os
import sys
import numpy as np
import pandas as pd
import torch
from collections import Counter, defaultdict
from transformers import AutoTokenizer, BasicTokenizer, AutoModelForMaskedLM
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.retrieval.evaluation import EvaluateRetrieval
from pyserini.analysis import Analyzer, get_lucene_analyzer
from tqdm import tqdm_notebook
import pytrec_eval

sys.path.append(os.pardir)
from splade_vocab.models  import Splade, BEIRSpladeModel, BEIRSpladeModelIDF

In [13]:
data_path = "/home/gaia_data/iida.h/BEIR/datasets/nfcorpus"
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

  0%|          | 0/3633 [00:00<?, ?it/s]

In [2]:
def search_result(model_path):
    model = Splade(model_path)
    model.eval()
    tokenizer = model.tokenizer
    beir_splade = BEIRSpladeModel(model, tokenizer)
    dres = DRES(beir_splade)
    retriever = EvaluateRetrieval(dres, score_function="dot")
    results = retriever.retrieve(corpus, queries)
    del retriever
    del dres
    del model
    return results

In [3]:
model_pathes = {"mlm-splade-71694": "/home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/",
               "mlm-splade-30522": "/home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_24-2022-04-11_23-21-18/",
                "splade-71694": "/home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model_init/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-init_model-raw-remove--71694-batch_size_40-2022-04-24_00-46-31/",
               "splade": "/home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_40-2022-05-01_12-37-20"}

In [4]:
tks = {"splade-71694": AutoTokenizer.from_pretrained(os.path.join(model_pathes["mlm-splade-71694"], "0_MLMTransformer")),
       "splade": AutoTokenizer.from_pretrained(os.path.join(model_pathes["splade"], "0_MLMTransformer")),
      "space": BasicTokenizer()}

# Embedding

In [11]:
def embed_move(ini_model, splade_model):
    mlm_head_vec = ini_model.cls.predictions.decoder.weight
    mlm_sphead_vec = splade_model.transformer.cls.predictions.decoder.weight
    mlm_head_vec_norm = mlm_head_vec / torch.unsqueeze(torch.norm(mlm_head_vec, dim=1), 1)
    mlm_sphead_vec_norm = mlm_sphead_vec / torch.unsqueeze(torch.norm(mlm_sphead_vec, dim=1), 1)
    l2 = torch.norm(mlm_head_vec - mlm_sphead_vec, dim=1)
    l2_org = torch.mean(l2[:30522])
    l2_add = torch.mean(l2[30522:])
    print(f"l2_org: {l2_org}, l2_add: {l2_add}")
    cos_sim = torch.sum(mlm_head_vec_norm * mlm_sphead_vec_norm, dim=1)
    cos_sim_org = torch.mean(cos_sim[:30522])
    cos_sim_add = torch.mean(cos_sim[30522:])
    print(f"cos_sim_org: {cos_sim_org}, cos_sim_add: {cos_sim_add}")
    topk_sim, topk_ids = torch.topk(-cos_sim, 50)
    print(-topk_sim)
    print(topk_ids)
    print(splade_model.tokenizer.decode(topk_ids))

In [16]:
mlm_splade_71694 = Splade(model_pathes["mlm-splade-71694"])
mlm_71694 = AutoModelForMaskedLM.from_pretrained("/home/gaia_data/iida.h/BEIR/model/pubmed_abst//bert-base-uncased/mlm_model/raw/remove/71694/")
embed_move(mlm_71694, mlm_splade_71694)
del mlm_splade_71694
del mlm_71694

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/
l2_org: 0.15013934671878815, l2_add: 0.1105366200208664
cos_sim_org: 0.9936464428901672, cos_sim_add: 0.9946423172950745
tensor([0.7846, 0.8510, 0.8895, 0.8917, 0.8968, 0.9234, 0.9273, 0.9407, 0.9428,
        0.9429, 0.9431, 0.9433, 0.9433, 0.9437, 0.9471, 0.9477, 0.9485, 0.9498,
        0.9520, 0.9539, 0.9542, 0.9542, 0.9550, 0.9551, 0.9552, 0.9552, 0.9552,
        0.9564, 0.9567, 0.9574, 0.9574, 0.9578, 0.9584, 0.9593, 0.9593, 0.9596,
        0.9597, 0.9599, 0.9600, 0.9603, 0.9603, 0.9613, 0.9615, 0.9616, 0.9618,
        0.9619, 0.9625, 0.9626, 0.9629, 0.9633], grad_fn=<NegBackward0>)
tensor([ 1010,  1996,  1012,  1011,  1998,  1999,  1000,  2008,  1037,  2030,
         2006,  2028,  1997,  2036,  2035,  1006,  2007,  2001,  2000,  1007,
 

In [17]:
mlm_splade = Splade(model_pathes["mlm-splade-30522"])
mlm_bert = AutoModelForMaskedLM.from_pretrained("/home/gaia_data/iida.h/BEIR/model/pubmed_abst//bert-base-uncased/mlm_model/raw/remove/30522/")
embed_move(mlm_bert, mlm_splade)
del mlm_bert
del mlm_splade

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_24-2022-04-11_23-21-18/
l2_org: 0.20315341651439667, l2_add: nan
cos_sim_org: 0.9896920323371887, cos_sim_add: nan
tensor([0.8513, 0.8847, 0.9096, 0.9107, 0.9143, 0.9148, 0.9336, 0.9387, 0.9422,
        0.9423, 0.9447, 0.9456, 0.9487, 0.9493, 0.9496, 0.9496, 0.9500, 0.9504,
        0.9512, 0.9524, 0.9533, 0.9534, 0.9535, 0.9542, 0.9548, 0.9552, 0.9554,
        0.9555, 0.9556, 0.9557, 0.9560, 0.9562, 0.9565, 0.9566, 0.9567, 0.9572,
        0.9572, 0.9573, 0.9574, 0.9575, 0.9578, 0.9578, 0.9579, 0.9582, 0.9582,
        0.9587, 0.9588, 0.9589, 0.9591, 0.9591], grad_fn=<NegBackward0>)
tensor([1010, 1996, 1011, 1998, 1000, 1012, 1999, 2001, 2018, 2008, 2249, 1037,
        1025, 2009, 2441, 2262, 2004, 2036, 2035, 2015, 2030, 2006, 2111, 2002,
        2012, 4725, 2143, 2

In [18]:
splade = Splade(model_pathes["splade"])
bert = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
embed_move(bert, splade)
del bert
del splade

path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


l2_org: 0.20133037865161896, l2_add: nan
cos_sim_org: 0.9893619418144226, cos_sim_add: nan
tensor([0.9144, 0.9246, 0.9302, 0.9306, 0.9345, 0.9430, 0.9469, 0.9476, 0.9480,
        0.9484, 0.9489, 0.9494, 0.9499, 0.9503, 0.9512, 0.9522, 0.9529, 0.9532,
        0.9543, 0.9543, 0.9544, 0.9546, 0.9549, 0.9549, 0.9551, 0.9552, 0.9552,
        0.9555, 0.9556, 0.9556, 0.9560, 0.9561, 0.9562, 0.9562, 0.9562, 0.9565,
        0.9565, 0.9566, 0.9568, 0.9570, 0.9572, 0.9576, 0.9577, 0.9577, 0.9578,
        0.9579, 0.9581, 0.9582, 0.9582, 0.9582], grad_fn=<NegBackward0>)
tensor([1996, 1010, 1000, 1998, 1012, 2249, 2001, 1025, 2002, 1011, 2008, 1999,
        2004, 2195, 2012, 2111, 2268, 2048, 2418, 3150, 2441, 2333, 2262, 4350,
        2456, 2053, 2164, 2036, 2526, 2329, 2006, 2371, 2225, 2021, 2030, 2009,
        2013, 2294, 3440, 2251, 2286, 2117, 2034,  999, 2061, 2081, 2506, 2238,
        2760, 2146])
the, " and. 2011 was ; he - that in as several at people 2009 two 2017 1980 opened moved 2012 s

In [18]:
all_results = {}
for k, model_path in model_pathes.items():
    results = search_result(model_path)
    all_results[k] = results

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/114 [00:00<?, ?it/s]

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_24-2022-04-11_23-21-18/


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/114 [00:00<?, ?it/s]

path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_40-2022-05-01_12-37-20


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/114 [00:00<?, ?it/s]

In [19]:
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'ndcg_cut_10'})
all_ndcg = {}
all_ave_ndcg = {}
for k, result in all_results.items():
    ndcg = evaluator.evaluate(result)
    all_ndcg[k] = ndcg
    all_ave_ndcg[k] = np.mean([val["ndcg_cut_10"] for val in ndcg.values()])

In [20]:
all_ave_ndcg

{'mlm-splade-71694': 0.34793522628074985,
 'mlm-splade-30522': 0.33529012462911234,
 'splade-71694': 0.34578516560691475,
 'splade': 0.3360477401299443}

In [48]:
df_all_ndcg = {}
model_names = ["mlm-splade-71694", "mlm-splade-30522", "splade-71694", "splade"]
for k in model_names:
    target = all_ndcg[k]
    df_all_ndcg[k] = [val['ndcg_cut_10'] for val in target.values()]
    
df_all_ndcg = pd.DataFrame(df_all_ndcg)
df_all_ndcg.head()

,mlm-splade-71694,mlm-splade-30522,splade-71694,splade
0,0.758692,0.772313,0.772313,0.780089
1,0.094567,0.105280,0.087076,0.154232
2,0.327307,0.394345,0.510254,0.518469
3,0.340985,0.439179,0.454814,0.393657
4,0.246906,0.118584,0.244641,0.167613


In [49]:
diff_from_splade = {}
diff_model_names = ["mlm-splade-71694", "mlm-splade-30522"]
q_index = list(queries.keys())
for k in diff_model_names:
    target = all_ndcg[k]
    base = all_ndcg["splade"]
    diff_from_splade[k] = [val1["ndcg_cut_10"] - val2["ndcg_cut_10"] for val1, val2 in zip(target.values(), base.values())]
    
df_diff_from_splade = pd.DataFrame(diff_from_splade)
df_diff_from_splade

,mlm-splade-71694,mlm-splade-30522
0,-0.021397,-0.007776
1,-0.059665,-0.048952
2,-0.191163,-0.124124
3,-0.052672,0.045522
4,0.079293,-0.049029
...,...,...
318,-0.051163,-0.153817
319,0.000000,0.000000
320,0.000000,0.046816
321,0.000000,0.081229


In [50]:
diff_from_mlm_splade = {}
diff_model_names = ["mlm-splade-71694", "splade"]
q_index = list(queries.keys())
for k in diff_model_names:
    target = all_ndcg[k]
    base = all_ndcg["mlm-splade-30522"]
    diff_from_mlm_splade[k] = [val1["ndcg_cut_10"] - val2["ndcg_cut_10"] for val1, val2 in zip(target.values(), base.values())]
    
df_diff_from_mlm_splade = pd.DataFrame(diff_from_mlm_splade)
df_diff_from_mlm_splade

,mlm-splade-71694,splade
0,-0.013621,0.007776
1,-0.010713,0.048952
2,-0.067039,0.124124
3,-0.098194,-0.045522
4,0.128322,0.049029
...,...,...
318,0.102654,0.153817
319,0.000000,0.000000
320,-0.046816,-0.046816
321,-0.081229,-0.081229


In [51]:
qids_adalm_splade_better = df_diff_from_splade[(df_diff_from_splade["mlm-splade-71694"] > 0.0)].index
qids_adalm_splade_better = [q_index[i] for i in qids_adalm_splade_better]

In [52]:
qids_adalm_splade_worse = df_diff_from_splade[df_diff_from_splade["mlm-splade-71694"] < 0.0].index
qids_adalm_splade_worse = [q_index[i] for i in qids_adalm_splade_worse]

In [53]:
def count_em_w_space_tk(model_name, qids, all_results):
    def encode_doc(model, e_query, text, t_query, t_query_space):
        t_text = model.tokenizer(text, max_length=512, return_tensors="pt")
        with torch.no_grad():
            t_text = {k: v.to(device) for k, v in t_text.items()}
            e_text = model.encode(**t_text)
            match_scores = e_query * e_text
            score, ids = torch.topk(match_scores, k=20)
            ids = ids[0][score[0] > 0.1]
            score = score[0][score[0] > 0.1]
            tids = model.tokenizer.decode(ids)
        t_tids = tids.split()
        match_type = {"em": [], "not-em": [], "space-tk": []}
        not_em = []
        for t in t_tids:
            if t in t_query:
                match_type["em"].append(t)
            else:
                match_type["not-em"].append(t)
         
            if t in t_query_space:
                match_type["space-tk"].append(t)
                
        return match_type
    
    device = torch.device("cuda")
    model_path = model_pathes[model_name]
    model = Splade(model_path)
    model.to(device)
    space_tk = BasicTokenizer()
    
    match_token_splade_nums = []
    expand_token_splade_nums = []
    match_token_tk_nums = []
        
    for qid in qids:
        query = queries[qid]
        top_10_doc = sorted(all_results[model_name][qid].items(), key=lambda x: -x[1])[:10]
        t_query_id = model.tokenizer(query, max_length=512, return_tensors="pt")
        t_query_id = {k: v.to(device) for k, v in t_query_id.items()}
        t_query = model.tokenizer.tokenize(query)
        t_query_space = space_tk.tokenize(query)
        with torch.no_grad():    
            e_query = model.encode(**t_query_id)

        top_qrels = []
        try:
            q_qrels = qrels[qid]
        except:
            continue
        for i, (cid, qrel) in enumerate(q_qrels.items()):            
            if qrel > 0:
                text = corpus[cid]["title"] + " " + corpus[cid]["text"]
                t_text = model.tokenizer.tokenize(text)
                match_token_tk_nums.append(len([t for t in t_query if t in t_text]))
                match_type = encode_doc(model, e_query, text, t_query, t_query_space)
                match_token_splade_nums.append(len(match_type["em"]))
                expand_token_splade_nums.append(len(match_type["not-em"]))
        

    return match_token_tk_nums, match_token_splade_nums, expand_token_splade_nums

In [23]:
observe_model = ["mlm-splade-71694", "mlm-splade-30522", "splade-71694", "splade"]
# observe_model = ["mlm-splade-62783", "splade"]
for model_name in observe_model:
    match_token_tk_nums, match_token_splade_nums, expand_token_splade_nums = count_em_w_space_tk(model_name, qids_adalm_splade_better, all_results)
    print(f"{model_name}, em_tk: {np.mean(match_token_tk_nums)}, em_splade: {np.mean(match_token_splade_nums)}, expand: {np.mean(expand_token_splade_nums)}")

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


mlm-splade-71694, em_tk: 0.7366015787287079, em_splade: 0.5675114250103864, expand: 3.2370170336518487
path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_24-2022-04-11_23-21-18/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


mlm-splade-30522, em_tk: 1.002492729538845, em_splade: 0.5446614042376402, expand: 1.8635230577482342
path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model_init/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-init_model-raw-remove--71694-batch_size_40-2022-04-24_00-46-31/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


splade-71694, em_tk: 0.7366015787287079, em_splade: 0.5191109264644787, expand: 2.8739094308267554
path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Token indices sequence length is longer than the specified maximum sequence length for this model (659 > 512). Running this sequence through the model will result in indexing errors


splade, em_tk: 1.002492729538845, em_splade: 0.4948068134607395, expand: 2.4941836310760284


In [24]:
observe_model = ["mlm-splade-71694", "mlm-splade-30522", "splade-71694", "splade"]
# observe_model = ["mlm-splade-62783", "splade"]
for model_name in observe_model:
    match_token_tk_nums, match_token_splade_nums, expand_token_splade_nums = count_em_w_space_tk(model_name, qids_adalm_splade_worse, all_results)
    print(f"{model_name}, em_tk: {np.mean(match_token_tk_nums)}, em_splade: {np.mean(match_token_splade_nums)}, expand: {np.mean(expand_token_splade_nums)}")

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


mlm-splade-71694, em_tk: 0.9676992321948636, em_splade: 0.740799576383373, expand: 3.0254169976171563
path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_24-2022-04-11_23-21-18/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


mlm-splade-30522, em_tk: 1.2306063012973258, em_splade: 0.6904951019327509, expand: 1.6025946518400846
path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model_init/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-init_model-raw-remove--71694-batch_size_40-2022-04-24_00-46-31/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


splade-71694, em_tk: 0.9676992321948636, em_splade: 0.6952607889859677, expand: 2.9780248874768334
path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors


splade, em_tk: 1.2306063012973258, em_splade: 0.691289383108287, expand: 2.2975906804342072


In [25]:
observe_model = ["mlm-splade-71694", "mlm-splade-30522", "splade-71694", "splade"]
# observe_model = ["mlm-splade-62783", "splade"]
for model_name in observe_model:
    match_token_tk_nums, match_token_splade_nums, expand_token_splade_nums = count_em_w_space_tk(model_name, q_index, all_results)
    print(f"{model_name}, em_tk: {np.mean(match_token_tk_nums)}, em_splade: {np.mean(match_token_splade_nums)}, expand: {np.mean(expand_token_splade_nums)}")

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


mlm-splade-71694, em_tk: 0.7456623966272093, em_splade: 0.5842386898005513, expand: 3.1595589427598507
path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_24-2022-04-11_23-21-18/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


mlm-splade-30522, em_tk: 0.9825685098102805, em_splade: 0.5404572725798605, expand: 1.6108318469271932
path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model_init/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-init_model-raw-remove--71694-batch_size_40-2022-04-24_00-46-31/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


splade-71694, em_tk: 0.7456623966272093, em_splade: 0.536322360953462, expand: 2.9527322847413653
path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors


splade, em_tk: 0.9825685098102805, em_splade: 0.5124858115777525, expand: 2.3431165882925247


# Check Sparsity

In [7]:
model = Splade(model_pathes["mlm-splade-71694"])
model = model.to(torch.device("cuda"))

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/


In [11]:
all_doc_len = []
for cid in tqdm_notebook(corpus):
    text = corpus[cid]["title"] + " " + corpus[cid]["text"]
    t_text_id = model.tokenizer(text, max_length=512, return_tensors="pt")
    for k in t_text_id:
        t_text_id[k] = t_text_id[k].to(torch.device("cuda"))
    with torch.no_grad():
        e_text = model.encode(**t_text_id)
    e_doc_len = torch.sum(e_text > 0)
    all_doc_len.append(e_doc_len.cpu().numpy())
    
print("mean: ", np.mean(all_doc_len))
print("var: ", np.var(all_doc_len))
print("std: ", np.std(all_doc_len))

/tmp/ipykernel_330048/2062905229.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for cid in tqdm_notebook(corpus):


  0%|          | 0/3633 [00:00<?, ?it/s]

mean:  291.7742912193779
var:  4079.336889292566
std:  63.86968677935227


In [12]:
analyzer = Analyzer(get_lucene_analyzer())
all_doc_len = []
vocab = set()
for cid in tqdm_notebook(corpus):
    text = corpus[cid]["title"] + " " + corpus[cid]["text"]
    t_text = analyzer.analyze(text)
    all_doc_len.append(len(t_text))
    vocab |= set(t_text)
    
print("mean: ", np.mean(all_doc_len))
print("var: ", np.var(all_doc_len))
print("std: ", np.std(all_doc_len))

/tmp/ipykernel_330048/1443639527.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for cid in tqdm_notebook(corpus):


  0%|          | 0/3633 [00:00<?, ?it/s]

mean:  175.47068538398017
var:  4273.928744286648
std:  65.37529154265125


# domain specificc words

In [5]:
def em_w_space_tk(model_name, qids, all_results):
    def encode_doc(model, e_query, text, t_query, t_query_space):
        t_text_id = model.tokenizer(text, max_length=512, return_tensors="pt")
        t_text = model.tokenizer.tokenize(text)
        with torch.no_grad():
            e_text = model.encode(**t_text_id)
            match_scores = e_query * e_text
            score, ids = torch.topk(match_scores, k=20)
            ids = ids[0][score[0] > 0.0]
            score = score[0][score[0] > 0.0]
            tids = model.tokenizer.decode(ids)
        t_tids = tids.split()
        match_type = {"em": [], "not-em": [], "space-tk": []}
        not_em = []
        for t in t_tids:
            if t in t_text:
                match_type["em"].append(t)
            else:
                match_type["not-em"].append(t)
         
            if t in t_query_space:
                match_type["space-tk"].append(t)
                
        return match_type
    
    model_path = model_pathes[model_name]
    model = Splade(model_path)
    space_tk = BasicTokenizer()
    
    match_token_splade = []
    expand_token_splade = []
    match_token_tk = []
        
    for qid in qids:
        query = queries[qid]
        top_10_doc = sorted(all_results[model_name][qid].items(), key=lambda x: -x[1])[:10]
        t_query_id = model.tokenizer(query, max_length=512, return_tensors="pt")
        t_query = model.tokenizer.tokenize(query)
        t_query_space = space_tk.tokenize(query)
        with torch.no_grad():    
            e_query = model.encode(**t_query_id)

        top_qrels = []
        try:
            q_qrels = qrels[qid]
        except:
            continue
        for i, (cid, qrel) in enumerate(q_qrels.items()):            
            if qrel > 0:
                text = corpus[cid]["title"] + " " + corpus[cid]["text"]
                t_text = model.tokenizer.tokenize(text)
                # match_token_tk_nums.append(len([t for t in t_query if t in t_text]))
                match_type = encode_doc(model, e_query, text, t_query, t_query_space)
                match_token_splade += match_type["em"]
                expand_token_splade += match_type["not-em"]
        

    return match_token_splade, expand_token_splade

In [6]:
def calc_idf(corpus, tokenizer):
    N = len(corpus)
    idf = defaultdict(float)
    df = Counter()
    for cid, doc in tqdm_notebook(corpus.items()):
        text = doc["title"] + " " + doc["text"]
        t_doc = tokenizer.tokenize(text)
        df.update(list(set(t_doc)))
    
    for v, freq in df.items():
        idf[v] = np.log(N/freq)
        df[v] = freq / N
    return df, idf

In [9]:
data_path = "/home/gaia_data/iida.h/BEIR/datasets/msmarco"
ms_corpus, ms_queries, ms_qrels = GenericDataLoader(data_folder=data_path).load(split="test")

  0%|          | 0/8841823 [00:00<?, ?it/s]

In [10]:
vocab_size = 71694
tk_names = ["splade", f"splade-{vocab_size}"]

In [16]:

ms_tk_df = {}
for model_name in tk_names:
    df, idf = calc_idf(ms_corpus, tks[model_name])
    ms_tk_df[model_name] = df

/tmp/ipykernel_1352878/3833919760.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for cid, doc in tqdm_notebook(corpus.items()):


  0%|          | 0/8841823 [00:00<?, ?it/s]

  0%|          | 0/8841823 [00:00<?, ?it/s]

In [19]:
print(ms_tk_df.keys())
print(tk_df.keys())

dict_keys(['splade', 'splade-71694'])
dict_keys(['splade', 'splade-71694'])


In [14]:
tk_df = {}
for model_name in tk_names:
    df, idf = calc_idf(corpus, tks[model_name])
    tk_df[model_name] = df

/tmp/ipykernel_1352878/3833919760.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for cid, doc in tqdm_notebook(corpus.items()):


  0%|          | 0/3633 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/3633 [00:00<?, ?it/s]

In [17]:
ms_tk_df[f"splade-{vocab_size}"] = ms_tk_df["splade-71684"]
tk_df[f"splade-{vocab_size}"] = tk_df["splade-71684"]

KeyError: 'splade-71684'

In [20]:
word_kind_model = {}

for model_name in tk_names:
    word_kind_model[model_name] = {"domain_words": [], "general_words": []}
    for v in tk_df[model_name]:
        if tk_df[model_name][v] > ms_tk_df[model_name].get(v, 0) * 5:
            word_kind_model[model_name]["domain_words"].append(v)
        else:
            word_kind_model[model_name]["general_words"].append(v)

In [21]:
def num_expand_words(model_name, expand_token_splade, word_kind_model):
    domain_words = []
    general_words = []
    if str(vocab_size) in model_name:
        for w in expand_token_splade:
            if w in word_kind_model[f"splade-{vocab_size}"]["domain_words"]:
                domain_words.append(w)
            else:
                general_words.append(w)
    else:
        for w in expand_token_splade:
            if w in word_kind_model["splade"]["domain_words"]:
                domain_words.append(w)
            else:
                general_words.append(w)
                
    print(f"expand-domain_words:{len(domain_words)}, expand-general_words:{len(general_words)}")

In [65]:
observe_model = [f"mlm-splade-{vocab_size}", "mlm-splade-30522", f"splade-{vocab_size}", "splade"]
# observe_model = ["mlm-splade-62783", "splade"]
for model_name in observe_model:
    match_token_splade, expand_token_splade = em_w_space_tk(model_name, qids_adalm_splade_better, all_results)
    print(model_name)
    num_expand_words(model_name, expand_token_splade, word_kind_model)

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


mlm-splade-71694
expand-domain_words:13588, expand-generad_words:12493
path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_24-2022-04-11_23-21-18/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


mlm-splade-30522
expand-domain_words:6096, expand-generad_words:6028
path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model_init/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-init_model-raw-remove--71694-batch_size_40-2022-04-24_00-46-31/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


splade-71694
expand-domain_words:10092, expand-generad_words:16738
path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Token indices sequence length is longer than the specified maximum sequence length for this model (659 > 512). Running this sequence through the model will result in indexing errors


splade
expand-domain_words:10613, expand-generad_words:11584


In [69]:
N = len(qids_adalm_splade_better)
print(N)
print(13588/N, 12493/N)
print(6096/N, 6028/N)
print(10092/N, 16738/N)
print(10613/N, 11584/N)

92
147.69565217391303 135.79347826086956
66.26086956521739 65.52173913043478
109.69565217391305 181.93478260869566
115.3586956521739 125.91304347826087


In [66]:
observe_model = [f"mlm-splade-{vocab_size}", "mlm-splade-30522", f"splade-{vocab_size}", "splade"]
# observe_model = ["mlm-splade-62783", "splade"]
for model_name in observe_model:
    match_token_splade, expand_token_splade = em_w_space_tk(model_name, q_index, all_results)
    print(model_name)
    num_expand_words(model_name, expand_token_splade, word_kind_model)

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


mlm-splade-71694
expand-domain_words:33794, expand-generad_words:30305
path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_24-2022-04-11_23-21-18/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


mlm-splade-30522
expand-domain_words:13629, expand-generad_words:12403
path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model_init/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-init_model-raw-remove--71694-batch_size_40-2022-04-24_00-46-31/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


splade-71694
expand-domain_words:24975, expand-generad_words:41281
path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_24-2022-04-07_21-45-37/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors


splade
expand-domain_words:24690, expand-generad_words:25786


In [106]:
df_diff_from_splade = pd.DataFrame(diff_from_splade)
df_diff_from_splade

,adalm,dapt-splade-59372,dapt-59372
0,0.105014,0.008879,0.099603
1,-0.157557,-0.157557,-0.157557
2,0.644442,0.545817,0.235547
3,-0.209543,-0.468651,-0.387676
4,0.087211,0.207809,0.094468
...,...,...,...
318,-0.085162,-0.032012,-0.021541
319,0.000000,0.000000,0.000000
320,-0.309806,-0.030386,0.021262
321,-0.243098,-0.131470,0.038757


In [125]:
df_diff_from_splade[(df_diff_from_splade["dapt-splade-59372"] < -0.15)]

,adalm,dapt-splade-59372,dapt-59372
1,-0.157557,-0.157557,-0.157557
3,-0.209543,-0.468651,-0.387676
19,-0.083009,-0.279622,-0.273529
22,-0.220092,-0.220092,-0.134949
25,0.039513,-0.151762,-0.078398
27,0.059688,-0.158731,0.005034
28,0.074855,-0.167160,-0.220258
34,-0.193092,-0.169580,-0.125174
44,-0.176018,-0.178651,0.063719
48,0.000000,-0.202774,-0.242394


In [119]:
df_diff_from_splade[(df_diff_from_splade["adalm"] > 0) & (df_diff_from_splade["dapt-splade-59372"] < -0.1)]
# df_diff_from_splade[(df_diff_from_splade["dapt-splade-59372"] < -0.15)]

,adalm,dapt-splade-59372,dapt-59372
9,0.161957,-0.139502,-0.139502
10,0.057063,-0.113103,-0.044399
25,0.039513,-0.151762,-0.078398
27,0.059688,-0.158731,0.005034
28,0.074855,-0.167160,-0.220258
64,0.445199,-0.330138,0.559816
75,0.258330,-0.112306,0.085143
168,0.059688,-0.114657,-0.098267
198,0.151320,-0.210811,-0.481900
200,0.002633,-0.538431,-0.227938


In [120]:
# qids_only_adalm_improve = [q_index[159], q_index[198], q_index[239], q_index[248]]
# qids_dapt_worse = [q_index[36], q_index[132], q_index[189], q_index[284]]
qids_dapt_worse = [q_index[3], q_index[189], q_index[280], q_index[306]]
qids_dapt_worse_adalm_better = [q_index[9], q_index[198], q_index[247], q_index[248]]

In [111]:
# df_diff_from_splade[(df_diff_from_splade["adalm"] < 0) & (df_diff_from_splade["dapt-59372"] > 0)]
df_diff_from_splade[(df_diff_from_splade["dapt-splade-59372"] > 0.15)]

,adalm,dapt-splade-59372,dapt-59372
2,0.644442,0.545817,0.235547
4,0.087211,0.207809,0.094468
26,0.110500,0.179477,0.220092
83,0.375670,0.507238,0.183410
115,0.286366,0.160385,0.106889
142,-0.169580,0.169580,-0.067483
193,0.173187,0.170108,0.188899
227,0.199183,0.298247,0.210534
240,0.914857,0.605095,0.542075
267,0.119021,0.252174,0.019907


In [1]:
def count_em_w_space_tk(model_name, qids, all_results):
    def encode_doc(model, e_query, text, t_query, t_query_space):
        t_text = model.tokenizer(text, max_length=512, return_tensors="pt")
        with torch.no_grad():
            e_text = model.encode(**t_text)
            match_scores = e_query * e_text
            score, ids = torch.topk(match_scores, k=20)
            ids = ids[0][score[0] > 0.1]
            score = score[0][score[0] > 0.1]
            tids = model.tokenizer.decode(ids)
        t_tids = tids.split()
        match_type = {"em": [], "not-em": [], "space-tk": []}
        not_em = []
        for t in t_tids:
            if t in t_query:
                match_type["em"].append(t)
            else:
                match_type["not-em"].append(t)
         
            if t in t_query_space:
                match_type["space-tk"].append(t)
                
        return match_type
    
    model_path = model_pathes[model_name]
    model = Splade(model_path)
    space_tk = BasicTokenizer()
    
    match_token_splade_nums = []
    expand_token_splade_nums = []
    match_token_tk_nums = []
        
    for qid in qids:
        query = queries[qid]
        top_10_doc = sorted(all_results[model_name][qid].items(), key=lambda x: -x[1])[:10]
        t_query_id = model.tokenizer(query, max_length=512, return_tensors="pt")
        t_query = model.tokenizer.tokenize(query)
        t_query_space = space_tk.tokenize(query)
        with torch.no_grad():    
            e_query = model.encode(**t_query_id)

        top_qrels = []
        try:
            q_qrels = qrels[qid]
        except:
            continue
        for i, (cid, qrel) in enumerate(q_qrels.items()):            
            if qrel > 0:
                text = corpus[cid]["title"] + " " + corpus[cid]["text"]
                t_text = model.tokenizer.tokenize(text)
                match_token_tk_nums.append(len([t for t in t_query if t in t_text]))
                match_type = encode_doc(model, e_query, text, t_query, t_query_space)
                match_token_splade_nums.append(len(match_type["em"]))
                expand_token_splade_nums.append(len(match_type["not-em"]))
        

    return match_token_tk_nums, match_token_splade_nums, expand_token_splade_nums

In [17]:
observe_model = ["mlm-splade-62783", "mlm-splade-30522", "splade-62783", "splade"]
# observe_model = ["mlm-splade-62783", "splade"]
for model_name in observe_model:
    match_token_tk_nums, match_token_splade_nums, expand_token_splade_nums = count_em_w_space_tk(model_name, qids_adalm_splade_better, all_results)
    print(f"{model_name}, em_tk: {np.mean(match_token_tk_nums)}, em_splade: {np.mean(match_token_splade_nums)}, expand: {np.mean(expand_token_splade_nums)}")

NameError: name 'count_em_w_space_tk' is not defined

In [112]:
# qid_only_dapt_59372_improve = q_index[232]
qids_dapt_59372_improve = [q_index[2], q_index[83], q_index[240], q_index[307]]

In [155]:
def analysis(model_name, qid, all_results):
    def encode_vec(model, e_query, text):
        t_text = model.tokenizer(text, max_length=512, return_tensors="pt")
        with torch.no_grad():
            e_text = model.encode(**t_text)
            match_scores = e_query * e_text
            score, ids = torch.topk(match_scores, k=20)
            ids = ids[0][score[0] > 0.1]
            score = score[0][score[0] > 0.1]
            tids = model.tokenizer.decode(ids)
        print([round(s, 2) for s in score.tolist()])
        print(tids)
    
    model_path = model_pathes[model_name]
    query = queries[qid]
    top_10_doc = sorted(all_results[model_name][qid].items(), key=lambda x: -x[1])[:10]
    model = Splade(model_path)
    t_query = model.tokenizer(query, max_length=512, return_tensors="pt")
    with torch.no_grad():    
        e_query = model.encode(**t_query)
    for i, (cid, score) in enumerate(top_10_doc):
        try:
            qrel = qrels[qid][cid]
        except:
            qrel = "not-eval"
        print(i, cid, score, qrel)
        text = corpus[cid]["title"] + " " + corpus[cid]["text"]
        encode_vec(model, e_query, text)
        
    del model
    

In [156]:
observe_model = ["adalm", "dapt-splade-59372", "dapt-59372", "splade"]
for qid in qids_dapt_worse:
    print("----------------------------")
    print(queries[qid])
    for model_name in observe_model:
        print("-------------")
        print(model_name)
        analysis(model_name, qid, all_results)

----------------------------
What’s Driving America’s Obesity Problem?
-------------
adalm
path /home/gaia_data/iida.h/BEIR/datasets/nfcorpus/new_model/adalm-bio/GenQ-adalm-bio-splade/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 MED-5006 96.15606689453125 not-eval
[6.29, 4.79, 3.97, 3.86, 3.76, 3.41, 3.22, 2.95, 2.86, 2.8, 2.68, 2.36, 2.17, 2.02, 1.96, 1.84, 1.82, 1.81, 1.74, 1.68]
obesity obese bmi overweight epidemic american adiposity us fatness prevalence weight usa americans america problem fat ow national cost south
1 MED-2723 87.42149353027344 2
[6.51, 4.82, 3.42, 3.24, 3.19, 3.01, 2.96, 2.95, 2.87, 2.51, 2.41, 2.14, 1.98, 1.96, 1.73, 1.69, 1.6, 1.49, 1.42, 1.41]
obesity obese epidemic adiposity weight bmi usa fatness overweight us prevalence american fat economic ow cost national osa mass south
2 MED-1328 80.39183044433594 not-eval
[6.43, 4.56, 4.01, 3.93, 3.28, 3.12, 2.91, 2.83, 2.64, 2.36, 2.34, 2.09, 1.88, 1.86, 1.84, 1.68, 1.59, 1.54, 1.51, 1.4]
obesity obese bmi overweight adiposity prevalence fatness epidemic weight country national ow mass american fat us south youth body android
3 MED-1448 77.73985290527344 not-eval
[6.48, 4.84, 3.54, 3.35, 3.15, 3.03, 2.46, 2.18, 1.98, 1.98, 1.95, 1.93, 1.9,

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 MED-1334 6.441590309143066 not-eval
[4.72, 1.57, 0.15]
obesity ’ heather
1 MED-1056 5.927430152893066 not-eval
[4.29, 1.64]
obesity ’
2 MED-1213 5.682057857513428 not-eval
[3.12, 1.48, 0.1]
obesity ’ s
3 MED-1790 5.591947078704834 not-eval
[3.73, 1.46, 0.38]
obesity ’ heather
4 MED-4766 4.918933868408203 not-eval
[4.92]
obesity
5 MED-4776 4.743581771850586 not-eval
[4.82]
obesity
6 MED-4768 4.7081298828125 not-eval
[4.71]
obesity
7 MED-1446 4.700077533721924 not-eval
[3.96, 0.38]
obesity ’
8 MED-1800 4.588093280792236 not-eval
[4.33]
obesity
9 MED-3052 4.565436363220215 not-eval
[4.71]
obesity
-------------
dapt-59372
path /home/gaia_data/iida.h/BEIR/datasets/nfcorpus/new_model/dapt/GenQ-dapt-splade-59372/


...

0 MED-1572 7.533722877502441 1
[5.28, 2.25]
fish toxin
1 MED-4961 7.4135565757751465 not-eval
[5.73, 1.67]
fish toxin
2 MED-5099 7.2167863845825195 not-eval
[7.06, 0.18]
fish neurotransmitter
3 MED-5098 7.214046001434326 not-eval
[7.21]
fish
4 MED-5100 7.190975666046143 not-eval
[7.18]
fish
5 MED-4736 7.174872398376465 not-eval
[7.27]
fish
6 MED-3012 7.082894325256348 not-eval
[7.0, 0.13]
fish neurotransmitter
7 MED-4959 6.904348850250244 not-eval
[5.58, 1.28, 0.3]
fish toxin neurotransmitter
8 MED-3029 6.808091640472412 not-eval
[6.82]
fish
9 MED-2907 6.808091640472412 not-eval
[6.82]
fish
-------------
dapt-59372
path /home/gaia_data/iida.h/BEIR/datasets/nfcorpus/new_model/dapt/GenQ-dapt-splade-59372/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 MED-4961 17.157421112060547 not-eval
[6.87, 5.73, 2.06, 0.57, 0.49, 0.42, 0.38, 0.33, 0.3]
fish toxin trout enterotoxin dogfish nicking wine rainbow hampshire
1 MED-4735 15.169953346252441 not-eval
[7.89, 2.38, 1.83, 1.18, 0.72, 0.56, 0.55]
fish trout aroma wine algae rainbow dogfish
2 MED-1572 15.060108184814453 1
[7.21, 3.64, 1.99, 1.05, 0.45, 0.34, 0.18, 0.14]
fish toxin trout algae enterotoxin dogfish tetracycline mycobacterium
3 MED-4360 14.987115859985352 not-eval
[5.27, 4.54, 2.84, 2.34]
toxin fish sexually transmitted
4 MED-5098 14.025197982788086 not-eval
[7.77, 2.45, 1.91, 0.94, 0.6, 0.5, 0.12]
fish trout aroma wine dogfish rainbow algae
5 MED-5100 13.960214614868164 not-eval
[7.79, 2.33, 1.09, 0.92, 0.77, 0.55, 0.51]
fish trout aroma wine algae dogfish rainbow
6 MED-2410 13.93433666229248 not-eval
[8.06, 2.48, 0.8, 0.69, 0.65, 0.65, 0.54]
fish trout aroma wine rainbow dogfish algae
7 MED-3028 13.923398971557617 not-eval
[7.89, 2.54, 0.88, 0.86, 0.84, 0.64, 0.46]
fish trout

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 MED-4360 26.49691390991211 not-eval
[3.89, 2.62, 2.59, 2.47, 2.44, 2.04, 2.03, 1.79, 1.23, 0.71, 0.66, 0.65, 0.49, 0.4, 0.36, 0.36, 0.35, 0.32, 0.21, 0.18]
fish sexually toxin transmitted sex fishes poison transmission cause toxic genus spread, fishing habitat drug chemical him virus food
1 MED-4966 21.171051025390625 2
[5.13, 2.45, 2.14, 2.03, 1.59, 1.58, 1.35, 0.76, 0.68, 0.56, 0.55, 0.37, 0.33, 0.32, 0.26, 0.21, 0.19, 0.19, 0.16, 0.15]
fish fishes poison sex sexually cause toxin transmitted habitat toxic fishing genus him, virus chemical from food is.
2 MED-1572 19.377138137817383 1
[5.31, 3.14, 2.35, 2.27, 1.47, 0.74, 0.65, 0.61, 0.44, 0.34, 0.33, 0.33, 0.31, 0.23, 0.21, 0.13, 0.12, 0.11]
fish toxin fishes poison cause toxic habitat fishing genus him chemical spread, virus from are. is
3 MED-1570 18.265485763549805 1
[4.05, 3.38, 2.13, 1.93, 1.48, 1.12, 0.72, 0.7, 0.47, 0.36, 0.34, 0.3, 0.21, 0.2, 0.19, 0.16, 0.14, 0.13]
fish toxin fishes poison cause toxic genus habitat fishing 

In [157]:
observe_model = ["adalm", "dapt-splade-59372", "dapt-59372", "splade"]
for qid in qids_dapt_worse_adalm_better:
    print("----------------------------")
    print(queries[qid])
    for model_name in observe_model:
        print("-------------")
        print(model_name)
        analysis(model_name, qid, all_results)

----------------------------
Stopping Heart Disease in Childhood
-------------
adalm
path /home/gaia_data/iida.h/BEIR/datasets/nfcorpus/new_model/adalm-bio/GenQ-adalm-bio-splade/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 MED-3255 74.21464538574219 1
[4.1, 3.93, 3.16, 2.32, 2.31, 2.26, 2.24, 2.09, 2.0, 1.89, 1.88, 1.81, 1.75, 1.73, 1.73, 1.63, 1.6, 1.57, 1.52, 1.46]
childhood children early child infancy intervention boys cvd infants pediatric atherosclerosis prevention age youth paediatric coronary heart cardiovascular adolescents ecg
1 MED-3979 63.191444396972656 not-eval
[4.46, 4.21, 3.04, 2.76, 2.46, 2.42, 2.36, 2.21, 2.21, 2.2, 1.95, 1.94, 1.85, 1.53, 1.43, 1.36, 1.35, 1.35, 1.33, 1.33]
childhood children child age infancy wheezing school paediatric early pediatric birth infants disease young adolescence year period pediatrics kindergarten youth
2 MED-3954 61.03766632080078 not-eval
[4.91, 4.41, 3.44, 2.79, 2.39, 2.35, 2.35, 2.33, 1.94, 1.93, 1.89, 1.86, 1.85, 1.79, 1.68, 1.57, 1.41, 1.35, 1.27, 1.23]
childhood heart children ihd cvd child cardiac disease boys ecg chd cardiovascular age early coronary bradycardia hr youth hearts atherosclerosis
3 MED-3253 60.666969299316406 2
[3.9, 3.0, 2.97, 2.8

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 MED-3954 11.270530700683594 not-eval
[3.9, 2.99, 2.9, 1.82]
heart childhood children disease
1 MED-1548 8.852556228637695 not-eval
[4.67, 2.42, 2.31]
heart disease children
2 MED-4919 7.52935791015625 not-eval
[3.17, 2.55, 2.08]
children disease childhood
3 MED-2590 7.197972774505615 not-eval
[4.46, 2.54, 0.2]
heart disease myocardium
4 MED-1366 6.953921318054199 not-eval
[3.65, 1.86, 1.45]
heart disease children
5 MED-1996 6.712478160858154 not-eval
[3.38, 2.07, 1.26]
children childhood disease
6 MED-2484 6.663737773895264 not-eval
[2.88, 2.54, 1.25]
children childhood disease
7 MED-2474 6.662322044372559 not-eval
[2.97, 2.78, 1.8]
children childhood disease
8 MED-1722 6.632974147796631 not-eval
[3.18, 2.06, 1.73]
children childhood disease
9 MED-2479 6.4849653244018555 not-eval
[2.77, 1.62, 1.01]
children disease childhood
-------------
dapt-59372
path /home/gaia_data/iida.h/BEIR/datasets/nfcorpus/new_model/dapt/GenQ-dapt-splade-59372/


...

0 MED-3954 10.787633895874023 not-eval
[3.95, 2.6, 2.59, 1.98]
heart children childhood disease
1 MED-1548 9.019461631774902 not-eval
[4.73, 2.64, 2.08, 0.19]
heart disease children baby
2 MED-4919 7.668177604675293 not-eval
[2.85, 2.78, 1.8, 0.41, 0.16]
children disease childhood baby long
3 MED-2590 7.661012649536133 not-eval
[4.51, 2.76, 0.15, 0.12, 0.11]
heart disease myocardium bacterial long
4 MED-1366 7.110385417938232 not-eval
[3.7, 2.02, 1.3]
heart disease children
5 MED-2479 6.958315849304199 not-eval
[2.49, 1.76, 0.88, 0.5, 0.35]
children disease childhood bacterial baby
6 MED-1722 6.721230506896973 not-eval
[2.86, 1.88, 1.78, 0.42, 0.15]
children disease childhood baby bacterial
7 MED-1996 6.688912391662598 not-eval
[3.03, 1.79, 1.37, 0.36, 0.12]
children childhood disease baby bacterial
8 MED-3425 6.665884017944336 not-eval
[3.6, 2.2, 0.31]
heart disease children
9 MED-4613 6.64697265625 1
[4.0, 2.49]
heart disease
-------------
dapt-59372
path /home/gaia_data/iida.h/BEIR/

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 MED-3954 18.944347381591797 not-eval
[5.05, 4.86, 4.48, 3.62, 0.34, 0.28, 0.24]
heart disease childhood children infancy age fever
1 MED-3979 14.423734664916992 not-eval
[5.3, 3.46, 3.43, 1.27, 0.87, 0.39, 0.13]
children childhood disease illness fever age infancy
2 MED-2590 14.417108535766602 not-eval
[5.26, 5.02, 2.34, 1.37, 0.44]
disease heart children myocardial cardiovascular
3 MED-1996 14.32397747039795 not-eval
[4.97, 4.28, 3.79, 1.0, 0.22]
children disease childhood fever age
4 MED-2484 13.659724235534668 not-eval
[4.68, 4.0, 3.58, 1.34]
childhood children disease fever
5 MED-2218 13.19542121887207 not-eval
[5.54, 4.36, 1.74, 0.96, 0.78, 0.46, 0.34]
disease heart children illness fever cardiovascular age
6 MED-1548 12.898365020751953 not-eval
[5.1, 4.21, 3.78, 1.16]
heart disease children cardiovascular
7 MED-1680 12.866908073425293 not-eval
[4.22, 2.93, 2.54, 1.97, 0.77, 0.21]
disease heart children myocardial cardiovascular age
8 MED-1987 12.627806663513184 not-eval
[5.07, 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 MED-3954 18.710073471069336 not-eval
[4.99, 3.64, 3.34, 2.55, 1.06, 1.01, 0.9, 0.76, 0.35, 0.13]
childhood heart disease child cause teen diseases hearts pre gene
1 MED-3253 18.670602798461914 2
[3.56, 3.33, 2.91, 2.11, 1.32, 0.97, 0.92, 0.75, 0.61, 0.56, 0.25, 0.13, 0.11]
heart childhood disease child start teen cause hearts diseases when begin gene early
2 MED-1949 16.173206329345703 not-eval
[3.71, 3.52, 2.64, 1.47, 1.4, 0.94, 0.89, 0.53, 0.34, 0.23, 0.14, 0.11]
disease heart child pre childhood cause diseases when teen start baby first
3 MED-1996 16.133638381958008 not-eval
[4.4, 3.06, 2.62, 1.62, 1.4, 1.23, 0.87, 0.4, 0.17, 0.11, 0.1]
childhood disease child heart teen cause diseases pre gene when early
4 MED-1994 15.264307022094727 not-eval
[3.67, 3.16, 2.47, 1.42, 1.28, 1.06, 1.02, 0.8, 0.14]
childhood disease child pre teen cause diseases heart gene
5 MED-2590 14.389830589294434 not-eval
[5.88, 5.01, 1.92, 1.27, 0.18]
heart disease hearts diseases cause
6 MED-4315 14.27379035

In [158]:
observe_model = ["adalm", "dapt-splade-59372", "dapt-59372", "splade"]
for qid in qids_dapt_59372_improve:
    print("----------------------------")
    print(queries[qid])
    for model_name in observe_model:
        print("-------------")
        print(model_name)
        analysis(model_name, qid, all_results)

----------------------------
How to Reduce Exposure to Alkylphenols Through Your Diet
-------------
adalm
path /home/gaia_data/iida.h/BEIR/datasets/nfcorpus/new_model/adalm-bio/GenQ-adalm-bio-splade/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 MED-2658 113.0926742553711 1
[5.96, 5.24, 4.23, 3.65, 3.58, 3.53, 2.77, 2.63, 2.48, 2.47, 2.46, 2.44, 2.33, 2.26, 1.95, 1.87, 1.86, 1.85, 1.76, 1.64]
alkylphenolphenol op nonyl phthalate alkyl chemicals phenol ap dehp edc bpa phenols bisphenol phenolic np endocrine compound pp
1 MED-2661 101.48649597167969 2
[5.48, 4.41, 4.08, 3.97, 3.55, 3.16, 3.13, 2.4, 2.39, 2.18, 2.13, 2.1, 2.04, 2.04, 2.02, 1.85, 1.66, 1.53, 1.51, 1.48]
alkylphenol opphenol nonyl phthalate ap alkyl dehp chemicals aps compound bpa np phenol phenols pp opp fin phenolic
2 MED-2651 95.25385284423828 2
[5.26, 4.38, 4.02, 3.85, 3.32, 3.16, 2.18, 2.14, 2.12, 2.12, 2.04, 1.98, 1.88, 1.86, 1.85, 1.79, 1.63, 1.62, 1.58, 1.57]
alkylp opphenolhenol nonyl phthalate phenols alkyl dehp phenol np chemicals ap phenolic oil compound opp ohp bpa pp
3 MED-118 95.25385284423828 2
[5.26, 4.38, 4.02, 3.85, 3.32, 3.16, 2.18, 2.14, 2.12, 2.12, 2.04, 1.98, 1.88, 1.86, 1.85, 1.79, 1.63, 1.62, 1.58, 1.57]
alkylp opphenolhenol nonyl phthala

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 MED-2644 10.95060920715332 2
[3.71, 2.83, 2.72, 0.79, 0.7, 0.12]
##phenolkyl alenol phenolss
1 MED-2658 10.930947303771973 1
[3.97, 2.91, 2.83, 0.59, 0.38, 0.18]
##phenol alkyl phenolsenols
2 MED-2661 10.169252395629883 2
[3.37, 2.87, 2.55, 0.44, 0.27, 0.18]
##phenolkyl al phenolsenols
3 MED-2651 9.294334411621094 2
[3.42, 2.54, 2.03, 0.44, 0.41, 0.32, 0.28]
##phenolkyl al phenols dietenol exposure
4 MED-118 9.294334411621094 2
[3.42, 2.54, 2.03, 0.44, 0.41, 0.32, 0.28]
##phenolkyl al phenols dietenol exposure
5 MED-2652 9.209200859069824 2
[3.06, 1.39, 1.35, 0.72, 0.42, 0.21, 0.15]
##phenol alkyl exposure phenolsenols
6 MED-2647 6.542079925537109 1
[2.76, 1.53, 0.87, 0.68, 0.37, 0.21, 0.12]
exposurephenol diet al phenolskyls
7 MED-1164 6.273993492126465 not-eval
[2.25, 1.63, 1.17, 1.1, 0.66]
exposure alkyl diet reduce
8 MED-1847 6.109560489654541 not-eval
[4.22, 1.0, 0.62, 0.12, 0.1]
alphenol phenolsenols
9 MED-1845 5.222037315368652 not-eval
[3.85, 1.29]
al diet
-------------
dapt-

...

0 MED-4095 6.3792572021484375 not-eval
[6.37]
cancer
1 MED-1564 6.053951740264893 1
[5.89]
cancer
2 MED-2814 5.992645263671875 not-eval
[6.08]
cancer
3 MED-3717 5.977495193481445 not-eval
[5.97]
cancer
4 MED-4227 5.9157209396362305 not-eval
[5.91]
cancer
5 MED-4070 5.903906345367432 not-eval
[6.02]
cancer
6 MED-1531 5.717670917510986 1
[5.99]
cancer
7 MED-1567 5.696262836456299 1
[5.76]
cancer
8 MED-1717 5.66422176361084 not-eval
[5.69]
cancer
9 MED-4096 5.6317315101623535 not-eval
[5.74]
cancer
-------------
dapt-59372
path /home/gaia_data/iida.h/BEIR/datasets/nfcorpus/new_model/dapt/GenQ-dapt-splade-59372/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 MED-3554 13.68480110168457 not-eval
[6.74, 2.14, 1.99, 0.88, 0.65, 0.26]
cancer ve carcinogenesis eosinophil malignancy anti
1 MED-2357 12.826347351074219 not-eval
[5.57, 2.95, 1.74, 1.68, 0.64, 0.37]
cancergg eosinophil carcinogenesis anti malignancy
2 MED-5326 11.714365005493164 not-eval
[7.11, 3.78, 0.73]
cancer carcinogenesis malignancy
3 MED-4030 11.640615463256836 not-eval
[7.58, 3.19, 0.77]
cancer carcinogenesis malignancy
4 MED-2247 11.636894226074219 not-eval
[6.76, 3.64, 0.86, 0.29]
cancer carcinogenesis malignancy anti
5 MED-3550 11.622136116027832 not-eval
[7.12, 3.0, 0.71, 0.67, 0.12]
cancer carcinogenesis malignancy anti perfringens
6 MED-4487 11.612100601196289 not-eval
[7.42, 3.43, 0.86, 0.11]
cancer carcinogenesis malignancy histories
7 MED-3703 11.597021102905273 not-eval
[7.08, 2.61, 1.33, 0.82]
cancer carcinogenesis eosinophil malignancy
8 MED-4070 11.524694442749023 not-eval
[7.41, 3.22, 0.8, 0.13, 0.11]
cancer carcinogenesis malignancy dysplasia histories
9 MED-

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 MED-4714 17.756385803222656 not-eval
[5.51, 3.29, 2.72, 2.03, 1.13, 1.03, 0.91, 0.89, 0.17]
cancer tumor food vegetables vegetable vs eat foods david
1 MED-1146 17.479660034179688 not-eval
[6.29, 3.64, 2.23, 1.65, 0.96, 0.92, 0.91, 0.64, 0.63, 0.19]
cancer tumor food vegetables vegetable eat difference david foods dana
2 MED-2075 17.17150115966797 not-eval
[5.66, 3.28, 2.44, 1.63, 1.11, 0.91, 0.85, 0.34, 0.1]
cancer tumor food vegetables foods vegetable eat david are
3 MED-890 17.123666763305664 not-eval
[5.92, 3.65, 2.6, 1.39, 1.27, 1.12, 0.65, 0.42]
cancer tumor food vegetables foods eat vegetable david
4 MED-3245 16.95833969116211 not-eval
[5.39, 2.96, 2.88, 1.79, 1.46, 1.07, 0.92]
cancer tumor food vegetables foods vegetable eat
5 MED-4861 16.85017204284668 not-eval
[5.81, 3.36, 2.24, 1.73, 1.02, 0.74, 0.62, 0.6]
cancer tumor food vegetables vegetable eat foods david
6 MED-4030 16.76387596130371 not-eval
[6.13, 3.45, 2.34, 1.31, 1.3, 1.01, 0.59, 0.46]
cancer tumor food foods vege